# Import

In [2]:
from geneticalgs import BinaryGA

import random

# import pandas as pd
import numpy as np
import csv
import time
import pickle

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
# Make the graphs a bit prettier, and bigger
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

# Common function definition

In [263]:
def save_to_pickle_file(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_pickle_file(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def generate_diff_weights(n):
    """Generates different random integers from interval (1, 5*n + 1)."""
    interval = (1, 5*n)
    
    random_number = random.randrange(interval[0], interval[1])
    used_values = [random_number]

    for i in range(1, n):
        while random_number in used_values:
            random_number = random.randrange(interval[0], interval[1])

        used_values.append(random_number)

    return used_values

def generate_weights(n):
    """Generates random integers (possible with duplicates) from interval (1, 5*n + 1)."""
    interval = (1, 5*n)
    w = []
    
    for i in range(n):
        w.append(random.randrange(interval[0], interval[1]))
        
    return w

def read_sat(filename, max_num):
    with open(filename, 'r') as src_file:
        counter = 0
        clause_list = []

        for line in src_file:
            if counter > max_num:
                break
            
            elems = line.strip().split(" ")

            if elems[0] == 'p':
                num_vars = int(elems[2])
                num_clauses = int(elems[-1])

#                 print('num_vars:', num_vars)
#                 print('num_clauses:', num_clauses)
                continue

            if elems[0] == 'c':
                continue

            if elems[0] == '%':
                break

    #         print(elems)
            clause = []
            # we have 3SAT and so we read three values
            for i in range(3):
                clause.append(int(elems[i]))
#             print(clause)

            clause_list.append(clause)
            counter += 1

#         if counter != num_clauses:
#             raise ValueError('The given amount of clauses is not equal to the specified one.')
            
        return clause_list, num_vars, counter - 1

# Read 3SAT in DIMACS CNF format

In [95]:
path = './data/uf50-218/'
filename = 'uf50-01.cnf'

sat_clauses, num_vars, num_clauses = read_sat(path + filename, 50*4)
print('num_vars:', num_vars)
print('num_clauses:', num_clauses)

# generate weights
weights = generate_diff_weights(num_vars)

num_vars: 50
num_clauses: 200


# Save to or load from a file with generated weights

In [264]:
# you may save or load generated weights
result_file = 'weights'

# save_to_pickle_file(weights, result_file)
# weights = load_pickle_file(result_file)

# Define fitness function and helper function for evaluating performance

This fitness function uses values readed from 3SAT.

In [170]:
def fitness_func(chromosome, data):
    score = 0  # amount of satisfied clauses
    active_vars = [data[idx] for idx in chromosome]  # vars evaluated to 1
    
    for clause in sat_clauses:
        evaluated = False
        
        for elem in clause:
            if not evaluated:
                if elem < 0 and abs(elem) not in active_vars:
                    score += 1
                    evaluated = True
                elif elem > 0 and abs(elem) in active_vars:
                    score += 1
                    evaluated = True
            else:
                break
                
#     print(score)
    sol_weight = 0
    if score == num_clauses:
        for var in active_vars:
            sol_weight += weights[var - 1]
            
    return score + sol_weight

def get_amount_of_solutions(population, already_found):
    # in case of fitness maximization
    sol_num = 0
#     already_found = []
    
    for i in range(-1, -len(population) - 1, -1):
        if population[i].fitness_val >= num_clauses:
            sorted_chromosome = list(population[i].chromosome)
            sorted_chromosome.sort()
#             print(10*'=')
#             print('already found:', already_found)
#             print('fitness:', population[i].fitness_val)
#             print('chromosome:', population[i].chromosome)
#             print('sorted chromosome:', sorted_chromosome)
            
            if sorted_chromosome not in already_found:
                already_found.append(sorted_chromosome)
                sol_num += 1
        else:
            break
            
    return sol_num, already_found

# Default GA parameters

In [296]:
optim = 'max'
elitism = True
input_data = list(range(1, num_vars + 1))

pop_size = 1000
r_sel = 'rank'
t_sel = 'tournament'
# tournament size
# whole population = pop_size * 3 -> this size will be adjusted to population size
t_size = [int(pop_size * 0.2), int(pop_size * 0.4), int(pop_size * 0.6), int(pop_size * 0.8), pop_size * 3]
mut_prob = 0 # 0.05
mut_type = 1
cross_prob = 1 # 0.95
cross_type = 1

generations = 20

# Function for running experiments

Body of this function (namely GA parameters) must be modified according to currently testing parameters. GA parameters are replaced with the best found values step by step.

In [307]:
def run_experiment(iter_num, gen_num, test_elems):
    """It is necessary to replace test parameter with the given test_elems"""
    result_found = [[] for x in test_elems]
    test_elem_num = len(test_elems)
    
    for it in range(iter_num):
        ga_mp = [[] for x in test_elems]

        for i in range(test_elem_num):
            ga_mp[i] = BinaryGA(input_data, fitness_func, optim=optim, selection='rank', 
                                mut_prob=0.01, mut_type=1, 
                                cross_prob=0.8, cross_type=3, 
                                elitism=elitism, tournament_size=None)
            ga_mp[i].init_random_population(2000)

        sols_mp = [[] for i in test_elems]
        already_found_mp = [[] for i in test_elems]

        for j in range(test_elem_num):
            sol_num, new_found_mp = get_amount_of_solutions(ga_mp[j].population, already_found_mp[j])
            already_found_mp[j] = list(new_found_mp)
            sols_mp[j].append(sol_num)

        for i in range(gen_num):
            for j in range(test_elem_num):
                ga_mp[j].run(1)

                sol_num, new_found_mp = get_amount_of_solutions(ga_mp[j].population, already_found_mp[j])
                already_found_mp[j] = list(new_found_mp)
                sols_mp[j].append(sol_num)

        for i in range(test_elem_num):
            result_found[i].append(len(already_found_mp[i]))
            
    return result_found

# Experiments

## Mutation probability

In [208]:
# mut_prob = [0.01, 0.05, 0.1, 0.15, 0.20]
# mut_prob = [0.008, 0.01, 0.012, 0.13, 0.15, 0.17]
mut_prob = [0.01, 0.15, 0.16, 0.17, 0.18]
iterations = 10

In [209]:
result_found = run_experiment(iterations, generations, mut_prob)

In [216]:
for i in range(len(mut_prob)):
    print(mut_prob[i], '=', np.mean(result_found[i]), result_found[i])

0.01 = 4.4 [1, 2, 3, 0, 0, 4, 23, 0, 0, 11]
0.15 = 0.3 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
0.16 = 0.4 [0, 0, 0, 0, 0, 3, 0, 0, 0, 1]
0.17 = 1.5 [0, 3, 0, 3, 0, 0, 1, 8, 0, 0]
0.18 = 2.0 [0, 0, 0, 0, 7, 0, 5, 0, 3, 5]


## Mutation type

In [229]:
# mut_type = [1, 2, 4, 6, 8, 10]
mut_type = [1, 5, 6, 7]
iterations = 10

In [230]:
result_found = run_experiment(iterations, generations, mut_type)

In [231]:
for i in range(len(mut_type)):
    print(mut_type[i], '=', np.mean(result_found[i]), result_found[i])

1 = 5.5 [1, 0, 0, 0, 34, 4, 4, 3, 8, 1]
5 = 1.6 [2, 0, 0, 0, 1, 0, 0, 11, 2, 0]
6 = 2.8 [0, 5, 5, 0, 6, 4, 1, 1, 4, 2]
7 = 1.9 [0, 0, 3, 0, 4, 3, 3, 4, 0, 2]


## Crossover probability

In [252]:
# cross_prob = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.97]
cross_prob = [0.73, 0.75, 0.77, 0.79, 0.8, 0.82]
iterations = 10

In [253]:
result_found = run_experiment(iterations, generations, cross_prob)

In [255]:
for i in range(len(cross_prob)):
    print(cross_prob[i], '=', np.mean(result_found[i]), result_found[i])

0.73 = 2.1 [1, 0, 0, 0, 0, 2, 0, 13, 2, 3]
0.75 = 3.3 [26, 1, 0, 1, 1, 0, 4, 0, 0, 0]
0.77 = 3.0 [1, 3, 0, 7, 0, 0, 10, 8, 1, 0]
0.79 = 1.4 [1, 0, 0, 0, 0, 12, 1, 0, 0, 0]
0.8 = 4.0 [0, 1, 0, 0, 24, 1, 6, 3, 0, 5]
0.82 = 1.3 [5, 0, 0, 0, 3, 2, 0, 3, 0, 0]


## Crossover type

In [266]:
cross_type = [1, 2, 3, 10, 20, 30, 40]
iterations = 10

In [267]:
result_found = run_experiment(iterations, generations, cross_type)

In [269]:
for i in range(len(cross_type)):
    print(cross_type[i], '=', np.mean(result_found[i]), result_found[i])

1 = 0.7 [6, 0, 0, 0, 0, 0, 0, 1, 0, 0]
2 = 1.0 [0, 2, 1, 0, 0, 4, 2, 1, 0, 0]
3 = 2.7 [0, 0, 0, 0, 1, 0, 6, 0, 0, 20]
10 = 2.5 [6, 3, 7, 0, 1, 4, 2, 1, 1, 0]
20 = 2.1 [0, 1, 11, 1, 3, 2, 2, 0, 0, 1]
30 = 4.4 [0, 28, 0, 6, 0, 7, 2, 0, 0, 1]
40 = 2.7 [0, 0, 2, 2, 2, 1, 2, 6, 3, 9]


## Selection type

In [298]:
selection = [r_sel, t_sel]
iterations = 10

In [287]:
result_found = run_experiment(iterations, generations, selection)

In [288]:
for i in range(len(selection)):
    print(selection[i], '=', np.mean(result_found[i]), result_found[i])

rank = 0.3 [0, 0, 1, 2, 0, 0, 0, 0, 0, 0]
tournament = 0.0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Change selection parameter in *run_experiment()* to 'tournament', change *tournament_size* and then run the following code

In [ ]:
result_found = run_experiment(iterations, generations, t_size)

In [ ]:
for i in range(len(selection)):
    print(selection[i], '=', np.mean(result_found[i]), result_found[i])

## Size of initial population

In [304]:
pop_size = [500, 1000, 1500, 2000]
iterations = 10

In [305]:
result_found = run_experiment(iterations, generations, pop_size)

In [306]:
for i in range(len(pop_size)):
    print(pop_size[i], '=', np.mean(result_found[i]), result_found[i])

500 = 0.1 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
1000 = 1.3 [0, 0, 0, 0, 2, 0, 0, 10, 1, 0]
1500 = 0.8 [0, 6, 0, 0, 0, 1, 0, 1, 0, 0]
2000 = 2.9 [14, 6, 0, 0, 5, 0, 0, 0, 0, 4]


# Run experiment for several different instances

In [ ]:
x = l

plt.plot(x, dictn4[40]['cross_prob'][0.3][0], '.', label='crossover prob. = 30%')
plt.plot(x, dictn4[40]['cross_prob'][0.6][0], '.', label='crossover prob. = 60%')
plt.plot(x, dictn4[40]['cross_prob'][0.95][0], '.', label='crossover prob. = 95%')
plt.xlabel('number of generation[int]')
plt.ylabel('avg. fitness value of generation [%]')
plt.legend(loc='best')
plt.title('Dependency between crossover probability and convergence speed')